In [1]:
import pandas as pd
#from google.colab import drive

In [2]:
file_path='/Users/jamie/Downloads/resto_reviews.csv'
resto_reviews= pd.read_csv(file_path)

In [7]:
resto_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319968 entries, 0 to 1319967
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Restaurant  1319968 non-null  object 
 1   Review      1317593 non-null  object 
 2   Rating      1319968 non-null  float64
dtypes: float64(1), object(2)
memory usage: 30.2+ MB


In [11]:
resto_reviews.drop_duplicates(inplace=True)

In [15]:
def word_count(review):
    return len(str(review).split())

# Apply the function to the 'review' column and count the reviews with less than 20 words
reviews_less_than_5_words = resto_reviews['Review'].apply(word_count) < 5
count_reviews = reviews_less_than_5_words.sum()

In [16]:
count_reviews

11267

In [17]:
# Get the indices of reviews with less than 5 words
indices_to_drop = resto_reviews[reviews_less_than_5_words].index

# Drop these reviews from the dataframe
resto_reviews.drop(indices_to_drop, inplace=True)

In [18]:
resto_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133349 entries, 0 to 1317826
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Restaurant  133349 non-null  object 
 1   Review      133349 non-null  object 
 2   Rating      133349 non-null  float64
dtypes: float64(1), object(2)
memory usage: 4.1+ MB


In [20]:
resto_reviews['Rating'].describe()

count    133349.000000
mean          3.593203
std           1.324065
min           1.000000
25%           3.000000
50%           4.000000
75%           5.000000
max           5.000000
Name: Rating, dtype: float64

In [ ]:
pip install spacy

In [5]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


for the problem of profermance, i choose a sub dataset for 100000 rows

In [21]:
import spacy

# Load a pre-trained NLP model from spaCy
nlp = spacy.load("en_core_web_sm")

def extract_entities(review_text):
    # Convert non-string data to string
    if not isinstance(review_text, str):
        review_text = str(review_text)
    doc = nlp(review_text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Apply the function to the 'Review' column after ensuring all data is of type str
resto_reviews['Entities'] = resto_reviews['Review'].astype(str).apply(extract_entities)


In [22]:
resto_reviews.head()

,Restaurant,Review,Rating,Entities
0,Jalsa,A beautiful place to dine in.The interiors tak...,4.0,"[(Mughal, ORG), (Christmas, DATE)]"
1,Jalsa,I was here for dinner with my family on a week...,4.0,"[(butter roti, PERSON)]"
2,Jalsa,Its a restaurant near to Banashankari BDA. Me ...,2.0,"[(Banashankari BDA, PERSON)]"
3,Jalsa,We went here on a weekend and one of us had th...,4.0,"[(a weekend, DATE), (one, CARDINAL), (two, CAR..."
4,Jalsa,The best thing about the place is itÃÃÃÃ...,5.0,"[(Second, ORDINAL)]"


In [23]:
resto_reviews['Recommend'] = resto_reviews['Rating'].apply(lambda x: 1 if x >= 4.0 else 0)

In [25]:
resto_reviews.to_csv('/Users/jamie/Downloads/resto_reviews_1.csv', index=False)

In [26]:
resto_reviews.head()

,Restaurant,Review,Rating,Entities,Recommend
0,Jalsa,A beautiful place to dine in.The interiors tak...,4.0,"[(Mughal, ORG), (Christmas, DATE)]",1
1,Jalsa,I was here for dinner with my family on a week...,4.0,"[(butter roti, PERSON)]",1
2,Jalsa,Its a restaurant near to Banashankari BDA. Me ...,2.0,"[(Banashankari BDA, PERSON)]",0
3,Jalsa,We went here on a weekend and one of us had th...,4.0,"[(a weekend, DATE), (one, CARDINAL), (two, CAR...",1
4,Jalsa,The best thing about the place is itÃÃÃÃ...,5.0,"[(Second, ORDINAL)]",1


In [27]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Load spaCy model
nlp = spacy.load("en_core_web_sm")


# Function to combine restaurant names and entities into a single string
def combine_features(row):
    # Extract and join entities from the tuple representation
    entities = ' '.join([ent[0] for ent in row['Entities']])
    return f"{row['Restaurant']} {entities}"

# Combine the features into a single column
resto_reviews['Features'] = resto_reviews.apply(combine_features, axis=1)

# Prepare the feature matrix X and target vector y
X = resto_reviews['Features']
y = resto_reviews['Recommend']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the models
rf_model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', RandomForestClassifier(random_state=42))
])

lr_model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(random_state=42))
])

svc_model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', SVC(probability=True, random_state=42))
])

# Train the models
rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)
svc_model.fit(X_train, y_train)

# Predict the results
rf_pred = rf_model.predict(X_test)
lr_pred = lr_model.predict(X_test)
svc_pred = svc_model.predict(X_test)

# Evaluate the models
print("Random Forest Classifier Report:")
print(classification_report(y_test, rf_pred))

print("Logistic Regression Classifier Report:")
print(classification_report(y_test, lr_pred))

print("Support Vector Classifier Report:")
print(classification_report(y_test, svc_pred))

/Applications/anaconda/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest Classifier Report:
              precision    recall  f1-score   support

           0       0.64      0.60      0.62     15773
           1       0.75      0.78      0.77     24232

    accuracy                           0.71     40005
   macro avg       0.70      0.69      0.69     40005
weighted avg       0.71      0.71      0.71     40005

Logistic Regression Classifier Report:
              precision    recall  f1-score   support

           0       0.65      0.54      0.59     15773
           1       0.73      0.81      0.77     24232

    accuracy                           0.70     40005
   macro avg       0.69      0.68      0.68     40005
weighted avg       0.70      0.70      0.70     40005

Support Vector Classifier Report:
              precision    recall  f1-score   support

           0       0.69      0.52      0.59     15773
           1       0.73      0.85      0.78     24232

    accuracy                           0.72     40005
   macro avg       0.7

In [28]:
from sklearn.ensemble import VotingClassifier

# Create the sub-models
estimators = [
    ('rf', rf_model), 
    ('lr', lr_model), 
    ('svc', svc_model)
]

# Create the ensemble model
ensemble = VotingClassifier(estimators, voting='soft')  # 'soft' if you want to predict class probabilities

# Train the ensemble model
ensemble.fit(X_train, y_train)

# Predict with the ensemble model
ensemble_predictions = ensemble.predict(X_test)

# Evaluate the ensemble model
print(classification_report(y_test, ensemble_predictions))

/Applications/anaconda/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.69      0.56      0.62     15773
           1       0.74      0.84      0.79     24232

    accuracy                           0.73     40005
   macro avg       0.72      0.70      0.70     40005
weighted avg       0.72      0.73      0.72     40005



In [31]:
import pickle

# Save the model to a file
with open('ensemble_model.pkl', 'wb') as file:
    pickle.dump(ensemble, file)


# Looks like the random forest have the best result

define a function that take the restuant from previous candidate generation precess, and extracte the entity form the user's interaction. we rank the the resturant list based the prediction of recommendation probability

In [34]:
import numpy as np
def rank_restaurants(model_pipeline, restaurants, sentence):
    # Extract entities from the sentence
    entities = extract_entities(sentence)
    
    # Combine the restaurant names with the entities
    combined_features = [f"{restaurant} {entities}" for restaurant in restaurants]
    
    # Initialize an empty list to hold probabilities
    probabilities = []
    
    # Iterate through each fitted classifier within the VotingClassifier
    for name, clf in model_pipeline.named_estimators_.items():
        # Check if the named classifier is a pipeline
        if hasattr(clf, 'named_steps'):
            # Transform features using the fitted vectorizer in the pipeline
            features_vectorized = clf.named_steps['vectorizer'].transform(combined_features)
            
            # Predict probabilities using the trained model in the pipeline
            clf_probabilities = clf.named_steps['classifier'].predict_proba(features_vectorized)[:, 1]
            
            # Append probabilities to the list
            probabilities.append(clf_probabilities)
    
    # Average the probabilities from all classifiers
    average_probabilities = np.mean(probabilities, axis=0)
    
    # Create a DataFrame with the results
    ranking_df = pd.DataFrame({
        'Restaurant': restaurants,
        'Entities': len(restaurants) * [entities],
        'Probability_of_Recommendation': average_probabilities
    })
    
    # Sort the DataFrame by probability of recommendation
    ranking_df.sort_values(by='Probability_of_Recommendation', ascending=False, inplace=True)
    
    return ranking_df


In [39]:
restaurant_list = ['The Boozy Griffin ', 'Chianti', 'Deja Vu Resto Bar','OPUS Club','Monkey Bar']
sentence = "recommend a italian pub"
ranked_df = rank_restaurants(ensemble, restaurant_list, sentence)

print(ranked_df)

           Restaurant           Entities  Probability_of_Recommendation
1             Chianti  [(italian, NORP)]                       0.846809
0  The Boozy Griffin   [(italian, NORP)]                       0.824016
2   Deja Vu Resto Bar  [(italian, NORP)]                       0.806264
3           OPUS Club  [(italian, NORP)]                       0.787903
4          Monkey Bar  [(italian, NORP)]                       0.779820
